In [4]:
import pandas as pd
from datetime import datetime 

import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

In [7]:
# Connect to the SQL database and open a cursor
import sqlite3 
conn = sqlite3.connect(r'C:\Users\theor\OneDrive\Desktop\NFL ANALYTICS\NFL_STATS_REFERENCE.db')
cursor = conn.cursor()

#Pull the dataset to mine 
# SQL query to select data
query = "SELECT * FROM ML_GAME_LEVEL_DATA_SET_FAV_VS_UD WHERE SEASON BETWEEN 2020 AND 2020;" 

# Use pandas to read the query result directly into a DataFrame
df_base = pd.read_sql_query(query, conn)

# Display the DataFrame
# List of all column names in the DataFrame
column_names = df_base.columns.tolist()
#print(column_names)

# Print the dataframe (optional)
#print(df_base)


# Format the current time as a string
current_time = datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(f"Step 1 completed: {formatted_time}") 

Step 1 completed: 2023-12-29 14:01:24


In [10]:
# Define the Dataframe
df_to_use = df_base

# Define the target variables to analyze 
target_name = 'RESULT_FAVORITE_TEAM_VICTORY'

# Define the feature variables to analyze 
feature_list = [
'DIV_GAME', 
'TEMPERATURE', 
'WINDSPEED', 
'OUTDOORS', 
'GRASS', 
'WEATHER_RAIN', 
'WEATHER_SNOW', 
'WEATHER_CLEAR', 
'WEATHER_WIND_RANK', 
'WEATHER_TEMPERATURE_RANK', 
'FAVORITE_TEAM_HOME_BINARY', 
'SPREAD_LINE', 
'TOTAL_LINE', 
'FAVORITE_TEAM_REST_DAYS', 
'UNDERDOG_TEAM_REST_DAYS', 
'FAVORITE_TEAM_REST_DAYS_DIFFERENTIAL_ACTUAL', 
'FAVORITE_TEAM_REST_DAYS_DIFFERENTIAL_BUCKET', 
'TREND_DIFF_FAVORITE_OV_VS_UNDERDOG_OV_PLAY_RANK', 
'TREND_DIFF_FAVORITE_OFF_VS_UNDERDOG_DEF_PLAY_RANK', 
'TREND_DIFF_FAVORITE_DEF_VS_UNDERDOG_OFF_PLAY_RANK', 
'TREND_DIFF_UNDERDOG_OV_VS_FAVORITE_OV_PLAY_RANK', 
'TREND_DIFF_UNDERDOG_OFF_VS_FAVORITE_DEF_PLAY_RANK', 
'TREND_DIFF_UNDERDOG_DEF_VS_FAVORITE_OFF_PLAY_RANK', 
'TREND_DIFF_FAVORITE_PASS_OFF_VS_UNDERDOG_PASS_DEF_PLAY_RANK', 
'TREND_DIFF_FAVORITE_RUSH_OFF_VS_UNDERDOG_RUSH_DEF_PLAY_RANK', 
'TREND_DIFF_UNDERDOG_PASS_OFF_VS_FAVORITE_PASS_DEF_PLAY_RANK', 
'TREND_DIFF_UNDERDOG_RUSH_OFF_VS_FAVORITE_RUSH_DEF_PLAY_RANK', 
'TREND_FAVORITE_OVERALL_AVG_PLAY_RANK', 
'TREND_UNDERDOG_KICKING_OFFENSE_FIELD_GOAL_MAKE_RATE_50'
]

# Specify the file path where you want to save the CSV file
export_file_path_string = r"C:\Users\theor\Python Scripts\Random_Forest_Focuser\results.csv"


In [11]:
target_df = df_to_use[[target_name]]
feature_df = df_to_use[['TREND_UNDERDOG_KICKING_OFFENSE_FIELD_GOAL_MAKE_RATE_50']]

In [42]:
def remove_nulls_and_non_numeric(target_df, feature_df):
    try:
        # Combine the dataframes
        combined_df = pd.concat([target_df, feature_df], axis=1) 
        original_length = len(combined_df)

        # Put the column names into variables 
        target_column_name = combined_df.columns[0]
        feature_column_name = combined_df.columns[1]

        # Initialize counters for non-numeric and null values in the columns
        non_numeric_count = {target_column_name: 0, feature_column_name: 0}
        null_count = {target_column_name: 0, feature_column_name: 0} 

        # Iterate through each column to count non-numeric and null values
        for column in combined_df.columns:
            # Convert to numeric, coercing non-numeric to NaN
            numeric_column = pd.to_numeric(combined_df[column], errors='coerce')

            # Counting non-numeric values (original non-NaN non-numeric values will become NaN)
            non_numeric_count[column] = numeric_column.isna().sum() - combined_df[column].isna().sum()

            # Counting null values (original NaN values)
            null_count[column] = combined_df[column].isnull().sum()

        # Print the counts of non-numeric and null values
        print("Non-numeric value count per column:")
        for col, count in non_numeric_count.items():
            print(f"{col}: {count}")

        print("\nNull value count per column:")
        for col, count in null_count.items():
            print(f"{col}: {count}")

        # Convert non-numeric values to NaN for each column
        for column in combined_df.columns:
            combined_df[column] = pd.to_numeric(combined_df[column], errors='coerce')

        # Drop rows with NaN values in either column
        combined_df.dropna(inplace=True) 

        # Get new length of dataframe 
        processed_length = len(combined_df) 
        print(f"The processed dataframe removed {original_length - processed_length} values with non-numeric or null values. The processed dataframe has {processed_length} data pairs.")

        # Return results 
        processed_target_df = combined_df[[target_column_name]].copy()
        processed_feature_df = combined_df[[feature_column_name]].copy() 
        return processed_target_df, processed_feature_df
        
    except Exception as e:
        print(f"There was an error when executing the remove_nulls_and_non_numeric() function. Error: {e}")



In [43]:
# Inital single variable loop
target_df = df_to_use[[target_name]]

for feature in feature_list:
    print(f"Processing feature: {feature}")
    
    # Create single column feature_df
    feature_df = df_to_use[[feature]]
    
    # Remove non-numeric and null values 
    cleanup_results = remove_nulls_and_non_numeric(target_df, feature_df)
    processed_target_df = cleanup_results[0] 
    processed_feature_df = cleanup_results[1] 
    print()
    print(processed_feature_df)
    print()
    

Processing feature: DIV_GAME
Non-numeric value count per column:
RESULT_FAVORITE_TEAM_VICTORY: 0
DIV_GAME: 0

Null value count per column:
RESULT_FAVORITE_TEAM_VICTORY: 0
DIV_GAME: 0
The processed dataframe removed 0 values with non-numeric or null values. The processed dataframe has 224 data pairs.

     DIV_GAME
0           0
1           1
2           1
3           0
4           0
..        ...
219         1
220         0
221         0
222         0
223         0

[224 rows x 1 columns]

Processing feature: TEMPERATURE
Non-numeric value count per column:
RESULT_FAVORITE_TEAM_VICTORY: 0
TEMPERATURE: 0

Null value count per column:
RESULT_FAVORITE_TEAM_VICTORY: 0
TEMPERATURE: 0
The processed dataframe removed 0 values with non-numeric or null values. The processed dataframe has 224 data pairs.

     TEMPERATURE
0           68.0
1           76.0
2           67.0
3           81.0
4           83.0
..           ...
219         68.0
220         68.0
221         36.0
222         46.0
223    